In [1]:
import torch
from torch._dynamo.utils import CompileProfiler

from transformers import WhisperForConditionalGeneration

from tqdm import tqdm

/opt/conda/envs/hf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en")
model.to("cuda").half();

In [3]:
BATCH_SIZE = 1
SEQ_LEN = 25
NUM_BATCHES = 100

# random input features - doesn't matter what they are since we'll generate to fix length
input_features = torch.randn((BATCH_SIZE, 80, 3000))
input_features = input_features.to("cuda").half()

In [4]:
# benchamrk vanilla generate
for i in tqdm(range(NUM_BATCHES)):
    pred_ids = model.generate(input_features, max_new_tokens=SEQ_LEN, min_new_tokens=SEQ_LEN)

100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:12<00:00,  8.23it/s]


In [5]:
profiler = CompileProfiler()
generate_fn = torch.compile(model.generate, backend=profiler)

# compilation step
pred_ids = generate_fn(input_features, max_new_tokens=SEQ_LEN, min_new_tokens=SEQ_LEN)

# benchmark compiled generate
for i in tqdm(range(100)):
    pred_ids = generate_fn(input_features, max_new_tokens=SEQ_LEN, min_new_tokens=SEQ_LEN)

100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:19<00:00,  5.00it/s]


In [6]:
# profiler report
print(profiler.report())

Torchdynamo Profiler Report

The following conditions caused torchdynamo to break out of tracing and fall back to python.
You may gain additional insight by passing `nopython=True` to torch._dynamo.optimize, to break on the first condition.
Graph Break Reason, Count
inline in skipfiles: GenerationMixin.generate  | decorate_context /opt/conda/envs/hf/lib/python3.10/site-packages/torch/utils/_contextlib.py, 1
call_function BuiltinVariable(str) [UserMethodVariable(<function WhisperForConditionalGeneration.prepare_inputs_for_generation at 0x7f6032ad1750>, NNModuleVariable())] {}, 3
call_function BuiltinVariable(repr) [NNModuleVariable()] {}, 2
call_method NNModuleVariable() _get_name [] {}, 2
call_function deepcopy in skip_files /opt/conda/envs/hf/lib/python3.10/copy.py, 1
call_method ConstDictVariable() copy [] {}, 1
call_function BuiltinVariable(set) [GetAttrVariable(InspectSignatureVariable(), parameters)] {}, 103
call_function UserDefinedClassVariable() [] {}, 4
call_method GetAttrVari